# Eksempel på multivariat data: FIFA-skillz
I dette eksempelet tar jeg for meg et datasett bestående av informasjon om alle spillere på FIFA 20, hentet fra [denne linken](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset). Antagelsen min er at mange ferdigheter er høyt korrelerte (er man god på langpasninger er man gjerne god på innlegg også, for eksempel), og dermed perfekt å anvende ulike metoder var multivariat analyse på.

## Datasett
Datasettet består av litt over 18 000 rad, der hver rad inneholder egenskapene til en fotballspiller. Hver kolonne svarer til en bestemt ferdighet. Etter å ha gjort et utvalg av hvilke ferdigheter (kolonner) som er relevante å analysere, sitter vi igjen med et datasett i form av en matrise $X$ som skriker etter å bli analysert.

## Avhengigheter og forberedende kode
Først importerer vi alle bibliotek vi trenger, forbereder plott, og definerer datasettet vårt. Her fjerner vi alle keepere fordi disse ikke har definert alle ferdigheter vi er interesserte i. Jeg har vært lite streng, og definert alle numeriske verdier som relevante. Dette betyr blant annet at absurde forklaringsvariabler som ``sofifa_id`` er med, men metodene som skal brukes burde uansett være robuste nok til å ignorere disse

In [200]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA

In [2]:
# Lag penere, mer høyoppløselige plott enn seaborn gjør med default-innstillinger
%config InlineBackend.figure_format = 'svg'
sns.set_context("notebook")
sns.set(style="ticks", font="Latin Modern Math")

In [7]:
datapath = "../datasett/fifa-20-complete-player-dataset/players_20.csv"
all_player_data = pd.read_csv(datapath)
all_player_data.sort_values('overall');
player_data_without_gk = all_player_data[all_player_data.loc[:, 'player_positions'] != 'GK']
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
player_data = player_data_without_gk.select_dtypes(include=numeric_dtypes)
player_data = player_data.dropna(axis='columns')

## Variabler
Vi er interesserte i å forklare totale ferdigheter, ``overall``, som funksjon av alle andre variabler. Disse blir da hhv. $y$ og $X$. Både ``overall``, og ``potential`` droppes fra $X$, siden disse henger så tett sammen. Jeg splitter datasettet opp i et test- og et treningssett, half ´n half.

In [195]:
player_data_train, player_data_test = train_test_split(player_data, test_size=0.2, random_state=1997)
y = player_data_train.loc[:, 'overall'].to_numpy()
y_test = player_data_test.loc[:, 'overall'].to_numpy()
X = player_data_train.drop(['overall', 'potential'], axis='columns').to_numpy()
X_test = player_data_test.drop(['overall', 'potential'], axis='columns').to_numpy()
n = len(y)
n_test = len(y_test)

In [199]:
mlr_fit = LinearRegression().fit(X, y)
y_hat = mlr_fit.predict(X)
y_hat_test = mlr_fit.predict(X_test)
mlr_mse_train = sum((y_hat - y)**2)/n
mlr_mse_test = sum((y_hat_test - y_test)**2)/n_test
print(f"MSE for prediksjoner på treningssett: {mlr_mse_train}")
print(f"MSE for prediksjoner på testsett: {mlr_mse_test}")

MSE for prediksjoner på treningssett: 4.64682588998231
MSE for prediksjoner på testsett: 4.815898694402428


Litt (men ikke særlig) avhengig av hvordan man deler opp i test- og treningssett ser vi at MSE for prediksjonene til denne estimatoren ligger på ca. $4,6$ for treningssett, og $4,8$ for testsett. Ikke spesielt imponerende, og heller ikke spesielt lovende når man tenker på at LS eksplisitt minimerer MSE for et gitt datasett.

Vi gir likevel ikke opp. Kanskje dataen kan si oss noe om hvilke variabler som best forklarer ``overall``? I så fall burde LS med $\ell_1$-regularisering (LASSO) gi sammenlignbare resultater som ren LS. Vi prøver.

TODO: forsøk mange ulike verdier av $\lambda$, og test på testsett.

In [234]:
lasso_fit = Lasso(alpha=10).fit(X, y)
y_hat = lasso_fit.predict(X)
y_hat_test = lasso_fit.predict(X_test)
mlr_mse_train = sum((y_hat - y)**2)/n
mlr_mse_test = sum((y_hat_test - y_test)**2)/n_test
print(f"MSE for prediksjoner på treningssett: {mlr_mse_train}")
print(f"MSE for prediksjoner på testsett: {mlr_mse_test}")
print(f"Antall ikke-null elementer i tilpasset modell: {lasso_fit.sparse_coef_.count_nonzero()}")

MSE for prediksjoner på treningssett: 9.140278446535188
MSE for prediksjoner på testsett: 9.609342104846986
Antall ikke-null elementer i tilpasset modell: 13
